# Модель прогнозирования спроса
Краткое описание:
<br>Необходимо создать алгоритм прогноза спроса на 14 дней для товаров собственного
производства. Гранулярность ТК-SKU-День.

Задача:
<br>Построить модель прогноза спроса на основе мастер данных и данных продаж с учетом разных
признаков.

Метрика качества:
<br>`WAPE`:
<br>`def wape(y_true: np.array, y_pred: np.array):`
    <br>`return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))`

### Описание данных
1) sales_df_train.csv –данные по продажам за скользящий год для обучения.
    * Столбцы:
        - st_id – захэшированное id магазина;
        - pr_sku_id – захэшированное id товара;
        - date – дата;
        - pr_sales_type_id – флаг наличия промо;
        - pr_sales_in_units – число проданных товаров всего (промо и без); **описание изменено после уточнения у бизнеса**
        - pr_promo_sales_in_units – число проданных товаров с признаком промо;
        - pr_sales_in_rub – продажи в РУБ всего (промо и без); **описание изменено после уточнения у бизнеса**
        - pr_promo_sales_in_rub – продажи с признаком промо в РУБ;
2) pr_df.csv – данные по товарной иерархии.
<br>От большего к меньшему pr_group_id - pr_cat_id - pr_subcat_id - pr_sku_id.
    - Столбцы:
        - pr_group_id – захэшированная группа товара;
        - pr_cat_id – захэшированная категория товара;
        - pr_subcat_id – захэшированная подкатегория товара;
        - pr_sku_id – захэшированное id товара;
        - pr_uom_id (маркер, обозначающий продаётся товар на вес или в ШТ).
3) pr_st.csv – данные по магазинам.
    - Столбцы:
        - st_id – захэшированное id магазина;
        - st_city_id – захэшированное id города;
        - st_division_code id – захэшированное id дивизиона;
        - st_type_format_id – id формата магазина;
        - st_type_loc_id – id тип локации/окружения магазина;
        - st_type_size_id – id типа размера магазина;
        - st_is_active – флаг активного магазина на данный момент.

## Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np

Пример файла с результатом прогноза

In [ ]:
# sales_submission_ex = pd.read_csv('sp_sales_task/sales_submission.csv')
# sales_submission_ex.head()

## Знакомство с данными

In [2]:
pr_df = pd.read_csv('sp_sales_task/pr_df.csv')
pr_st = pd.read_csv('sp_sales_task/st_df.csv')

In [3]:
display(pr_df.sample(1))
pr_df.info()
display(pr_st.sample(1))
pr_st.info()

,pr_sku_id,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id
412,5e664d6988c6d6827cbaaafb0bc7f103,aab3238922bcc25a6f606eb525ffdc56,3de2334a314a7a72721f1f74a6cb4cee,7d45a7d177df53e95d433b20bddcd073,17


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pr_sku_id     2050 non-null   object
 1   pr_group_id   2050 non-null   object
 2   pr_cat_id     2050 non-null   object
 3   pr_subcat_id  2050 non-null   object
 4   pr_uom_id     2050 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 80.2+ KB


,st_id,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active
10,62f91ce9b820a491ee78c108636db089,1587965fb4d4b5afe8428a4a024feb0d,81b4dd343f5880df806d4c5d4a846c64,4,3,32,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   st_id              12 non-null     object
 1   st_city_id         12 non-null     object
 2   st_division_code   12 non-null     object
 3   st_type_format_id  12 non-null     int64 
 4   st_type_loc_id     12 non-null     int64 
 5   st_type_size_id    12 non-null     int64 
 6   st_is_active       12 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 800.0+ bytes


#### Открытие файла с распознаванием дат и формированием новых индексов

In [4]:
sales_dt = pd.read_csv('sp_sales_task/sales_df_train.csv', index_col = [2], parse_dates = [2])
sales_dt.sort_index(inplace=True)
sales_dt.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 883015 entries, 2022-08-01 to 2023-07-18
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    883015 non-null  object 
 1   pr_sku_id                883015 non-null  object 
 2   pr_sales_type_id         883015 non-null  int64  
 3   pr_sales_in_units        883015 non-null  float64
 4   pr_promo_sales_in_units  883015 non-null  float64
 5   pr_sales_in_rub          883015 non-null  float64
 6   pr_promo_sales_in_rub    883015 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 53.9+ MB


In [5]:
sales_dt.tail()

,st_id,pr_sku_id,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub
date,,,,,,,
2023-07-18,6364d3f0f495b6ab9dcf8d3b5c6e0b01,e6449990bb6a761a964e58dd95f7a479,0,2.0,0.0,225.0,0.0
2023-07-18,42a0e188f5033bc65bf8d78622277c4e,eb341a778d385ad6ebe16e90efb48c08,1,10.0,10.0,2291.0,2291.0
2023-07-18,42a0e188f5033bc65bf8d78622277c4e,d362736d288d6997016766bfdf846693,1,1.0,1.0,97.0,97.0
2023-07-18,6364d3f0f495b6ab9dcf8d3b5c6e0b01,22b3872c9768b6070e1111d85e8e9660,0,0.0,0.0,106.0,0.0
2023-07-18,c81e728d9d4c2f636f067f89cc14862c,a7a9eb3ffb9634e37c50995c34da34d0,0,5.0,0.0,250.0,0.0


#### Возможны ли продажи по регулярной цене в промо-день

In [6]:
sales_dt['price_difference'] = sales_dt['pr_promo_sales_in_rub'] - sales_dt['pr_sales_in_rub']
sales_dt_group_by_sales_type = (
    sales_dt
    .groupby('pr_sales_type_id')
    .agg({'pr_sales_in_rub':'sum', 'pr_promo_sales_in_rub':'sum', 'price_difference':'sum'})
)
sales_dt_group_by_sales_type.head()

,pr_sales_in_rub,pr_promo_sales_in_rub,price_difference
pr_sales_type_id,,,
0,266616330.0,0.0,-266616330.0
1,260283295.0,260283295.0,0.0


**Вывод:** сумма продаж в день с промо полность равна общей сумме продаж в этот день. Возможно удаление признаков, связанных с промо.

#### Проверка пустых значений в количестве

In [7]:
sales_units_zero = sales_dt[sales_dt['pr_sales_in_units'] == 0]
print(
    'нулевые продажи в шт и руб:', 
    sales_units_zero[sales_units_zero['pr_sales_in_rub'] == 0]['pr_sales_in_rub'].count(), 
    '\nих доля в данных:', 
    round(sales_units_zero[sales_units_zero['pr_sales_in_rub'] == 0]['pr_sales_in_rub'].count() / sales_dt['pr_sales_in_rub'].count() * 100, 2), '%'
)
print(
    'нулевые продажи в шт, но не нулевые в рублях:', 
    sales_units_zero[sales_units_zero['pr_sales_in_rub'] != 0]['pr_sales_in_rub'].count(), 
    '\nих доля в данных:', 
    round(sales_units_zero[sales_units_zero['pr_sales_in_rub'] != 0]['pr_sales_in_rub'].count() / sales_dt['pr_sales_in_rub'].count() * 100, 2), '%'
)
sales_units_zero[sales_units_zero['pr_sales_in_rub'] != 0].head()

нулевые продажи в шт и руб: 454 
их доля в данных: 0.05 %
нулевые продажи в шт, но не нулевые в рублях: 66089 
их доля в данных: 7.48 %


,st_id,pr_sku_id,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,price_difference
date,,,,,,,,
2022-08-01,fa7cdfad1a5aaf8370ebeda47a1ff1c3,3b9ff9697e5688d38d5a0e7b0f0d519e,0,0.0,0.0,59.0,0.0,-59.0
2022-08-01,fa7cdfad1a5aaf8370ebeda47a1ff1c3,f28e2941cd8ffdf35d778c3578cf7041,0,0.0,0.0,83.0,0.0,-83.0
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,706b9e39dd3ca40669b5f5c74bfebeb8,0,0.0,0.0,76.0,0.0,-76.0
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,04bbb07b1057b09d04209991f3eadd8f,1,0.0,0.0,85.0,85.0,0.0
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,90702dbda20da8380fb559b1ea8c0140,0,0.0,0.0,98.0,0.0,-98.0


**Вывод:** 
- в данных имеются записи, которые можно удалить (их доля 0.05%): нулевые значения в количестве проданных товаров и их стоимости
- записи, по которым требуется уточнение в бизнесе (их доля 7.48%): нулевые значения в количестве проданных товаров и ненулевые в их стоимости.

#### Количество активных магазинов

In [8]:
print('количество магазинов без маркера активности:', 
      pr_st[pr_st['st_is_active'] == 0]['st_id'].count(), 
      '\nих доля в общем количестве магазинов:', 
      round(pr_st[pr_st['st_is_active'] == 0]['st_id'].count() / pr_st['st_id'].count(), 2)
     )

количество магазинов без маркера активности: 2 
их доля в общем количестве магазинов: 0.17


#### Количество данных по неактивным магазинам

In [9]:
unactive_store_sales = pr_st[pr_st['st_is_active'] == 0].merge(sales_dt, on='st_id', how='left')
print(
    'количество записей по неактивным магазинам:', 
    unactive_store_sales['pr_sales_in_rub'].count(), 
    '\nих доля в данных:', 
    round(unactive_store_sales['pr_sales_in_rub'].count() / sales_dt['pr_sales_in_rub'].count() * 100, 2), '%'
)

количество записей по неактивным магазинам: 729 
их доля в данных: 0.08 %


**Вывод:** в данных найдены магазины без маркера активности. Возможно, их можно удалить, требуется уточнение у бизнеса.

### Выводы
- Данные загружены с распознованием дат.
- Данные предоставлены за период с `01.08.2022` по `18.07.2023`
- Можем удалить признаки, связанные с промо, т.к. они дублируют общие значения
- Можно удалить записи (около 7.5% датасета):
    - с нулевыми значениями в количестве проданных товаров и их стоимости,
    - с нулевыми значениями в количестве проданных товаров и ненулевыми в их стоимости (действие согласовано с бизнесом)
- Можно удалить магазины без маркера активности (действие согласовано с бизнесом):
    - им соответствует 729 записей, что составляет 0.08% датасета

## Предобработка данных

#### Удаление данных, связанных с промо

In [26]:
sales_dt = sales_dt.drop(['pr_sales_type_id', 'pr_promo_sales_in_units', 'pr_promo_sales_in_rub', 'price_difference'], axis=1)
sales_dt.head()

,st_id,pr_sku_id,pr_sales_in_units,pr_sales_in_rub
date,,,,
2022-08-01,42a0e188f5033bc65bf8d78622277c4e,5781a2637b476d781eb3134581b32044,2.0,463.0
2022-08-01,c81e728d9d4c2f636f067f89cc14862c,82704fa1c2dbdb928bf4eed0667260dd,2.0,204.0
2022-08-01,c81e728d9d4c2f636f067f89cc14862c,0cc8850f66397af21700e3c060a5210c,9.0,639.0
2022-08-01,fa7cdfad1a5aaf8370ebeda47a1ff1c3,c4a665596d4f67cecb7542c9fad407ee,13.0,1734.0
2022-08-01,c81e728d9d4c2f636f067f89cc14862c,dce1f234d6424aa61f8e7ce0baffd9af,2.0,313.0


#### Удаление данных с нулями в количестве

In [35]:
sales_dt = sales_dt.loc[sales_dt['pr_sales_in_units'] != 0]
sales_dt.info()
sales_dt.sample()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 816472 entries, 2022-08-01 to 2023-07-18
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   st_id              816472 non-null  object 
 1   pr_sku_id          816472 non-null  object 
 2   pr_sales_in_units  816472 non-null  float64
 3   pr_sales_in_rub    816472 non-null  float64
dtypes: float64(2), object(2)
memory usage: 31.1+ MB


,st_id,pr_sku_id,pr_sales_in_units,pr_sales_in_rub
date,,,,
2022-09-18,c81e728d9d4c2f636f067f89cc14862c,c24c65259d90ed4a19ab37b6fd6fe716,48.0,8174.0


#### Удаление данных по неактивным магазинам

In [49]:
unactive_st = pr_st[pr_st['st_is_active'] == 0]
active_pr_st = pr_st.loc[pr_st['st_is_active'] != 0]

sales_dt = sales_dt.loc[~sales_dt['st_id'].isin(list(unactive_st['st_id']))]

#### Импорт календаря РФ

In [14]:
calendar = pd.read_csv('holidays_covid_calendar.csv')
calendar.info()
calendar.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   year     3653 non-null   int64 
 1   day      3653 non-null   int64 
 2   weekday  3653 non-null   int64 
 3   date     3653 non-null   object
 4   calday   3653 non-null   int64 
 5   holiday  3653 non-null   int64 
 6   covid    3653 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 199.9+ KB


,year,day,weekday,date,calday,holiday,covid
977,2017,4,1,04.09.2017,20170904,0,0


In [51]:
# list(unactive_st['st_id'])
# test_1 = sales_dt
# test_1 = test_1.loc[~test_1['st_id'].isin(list(unactive_st['st_id']))]
# test_1.info()
# test_1.head()
# sales_dt.info()
# sales_dt.head()

In [52]:
# calendar['covid'].value_counts()

## Чек-лист
1. Файл в зафиксированном формате с результатом прогноза спроса (sales_submission.csv).
2. Воспроизводимый код на Python
3. Описание решения:
    
    a. Описание обученной модели прогноза спроса
    
        i. Признаки
        ii. интерпретация (shapley values),
        iii. кросс-валидация
        iv. алгоритмы
    
    b. Описание вашего алгоритма оптимизации:
    
        i. методология расчетов
        ii. скорость оптимизации